# 背景

对指定个股和指数进行30分钟级别的监控，包括趋势、反转指标等，并通过语音进行报告。

In [1]:
! export HTTPS_PROXY=""
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
from omicron.models.stock import Stock
import datetime
import itertools
from IPython.display import display
await init_notebook(adaptor='both')

g = {
    "notified": set(),
}


In [ ]:
def get_pmae_threshold(code, frame_type, win):
    if code in ["000001.XSHG"]:
        return {
            "30m": {
                5: 1.51e-4,
                10: 1.51e-4
            }
        }.get(frame_type, {}).get(win, 3e-4)
    else: # 个股
        return {
            "30m": {
                5: 3e-3,
                10: 1e-3
            }
        }.get(frame_type, {}).get(win, 3e-3)
    
def pred_roc(close, win, pmae_threshold):
    """通过ma来预测2周期后的涨跌幅"""
    preds, pmae = predict_by_moving_average(close, win, n_preds = 2, err_threshold = pmae_threshold, n = 7)
    
    if preds:
        return preds[-1] / close[-1] - 1
    else:
        return None
    
def notify(frame, text):
    key = f"{frame.hour:02d}:{frame.minute:02d} {text}"
    if key in g["notified"]:
        return
    
    print(key)
    g["notified"].add(key)
    nb_say(text)
        
def canonical_code(code):
    if "." in code:
        return code
    
    if code.startswith("6"):
        return code + ".XSHG"
    else:
        return code + ".XSHE"
    
def to_code(name):
    return Stock._stocks[Stock._stocks["display_name"] == name]["code"][0]

async def day_rsi_proba(code, tm=None):
    end = arrow.get(tm).date() if tm else arrow.now().date()
    start = tf.day_shift(end, -59)
    
    bars = await Stock(code).load_bars(start, end, FrameType.DAY)
    bars = bars[np.isfinite(bars["close"])]
    close = bars["close"]
    
    rsi = relative_strength_index(close, 6)[-2:]
    return [round(rsiday.get_proba(code, rsi[i]),2) for i in range(2)]

def u_turn(bars):
    n = len(bars)
    half = n // 2
    
    c = bars["close"]
    o = bars["open"]
    
    # 前半程以收阴为主，后半程全部收阳
    flag = bars["close"] > bars["open"]
    if np.all(flag[-half:]) and np.all(~flag[:(n-half)]):
        if c[-1] > c[0]:
            return 'U'
    
    # 后半程全部收阴，前半程收阳为主
    if np.all(~flag[-half:]) and np.all(flag[:(n-half)]):
        if c[-1] < c[0]:
            return '^'
        
    return ""

In [ ]:
async def do_check(holdings:List[str], candidates, check_at=None):
    end = arrow.now() if check_at is None else arrow.get(check_at)
    start = tf.shift(tf.floor(end, FrameType.MIN30), -39, FrameType.MIN30)
    
    for code in itertools.chain(["000001.XSHG"], holdings, candidates):
        sec = Stock(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.MIN30)
        except Exception as e:
            print(e)
            continue
            
        bars = bars[np.isfinite(bars["close"])]
        close = bars["close"]
        
        rsi = relative_strength_index(close, period=6)[-1]
        prsi = rsi30.get_proba(code, rsi)
        
        print(f"{sec.display_name} {prsi:.0f}")
        
        if code == "000001.XSHG":
            notify(end, f"沪指相对强弱指标位置{prsi:.02f}.")
        if prsi >= 0.9 and code in ["000001.XSHG", *holdings]:
            notify(end, f"相对强弱指标显示，{sec.display_name}即将高位反转，概率为{prsi:.0%}.")
        if prsi <= 0.1 and code in ["000001.XSHG", *candidates]:
            notify(end, f"相对强弱指标显示，{sec.display_name}即将低位反转，概率为{1-prsi:.0%}.")
    
async def mon(holdings, candidates, check_at=None):
    import asyncio
    
    holdings = [to_code(name) for name in holdings]
    candidates = [to_code(name) for name in candidates]
    
    now = arrow.now()
    wakeup_time = []
    for tm in ["10:00", "10:15", "10:30", "10:45", "11:00", "11:15", "11:30", "13:15", "13:30",
              "13:45", "14:00", "14:15", "14:30", "14:45"]:
        hour, minute = map(int, tm.split(":"))
        wakeup_time.append(arrow.Arrow(now.year, now.month, now.day, hour, minute, tzinfo="Asia/Shanghai"))
        
    if check_at:
        wakeup_time.insert(0, arrow.now().shift(seconds=5))
        
    end = tf.floor(arrow.now(), FrameType.DAY)
    start = tf.day_shift(end, -40)
    
    data = []
    for code in itertools.chain(["000001.XSHG"], holdings, candidates):
        sec = Stock(code)
        bars = await sec.load_bars(start, end, FrameType.DAY)
        bars = bars[np.isfinite(bars["close"])]
        
        close = bars["close"]
        rsi = relative_strength_index(close, 6)[-3:]
        prsi = [round(rsiday.get_proba('000001.XSHG', r), 2) for r in rsi]

        data.append([sec.display_name, *prsi])
        
    df = pd.DataFrame(data, columns=["name", "rsi0", "rsi1", "rsi2"])
    display(df)
    
    for tm in wakeup_time:
        if arrow.now() > tm:
            continue
            
        seconds = (tm.timestamp - arrow.now().timestamp)
        await asyncio.sleep(seconds)
        print(f"=========== {tm.hour}:{tm.minute:02d}============")
        await do_check(holdings, candidates, check_at)

In [ ]:
holdings = [
    "中国西电", 
    "格林美", 
    "锦盛新材",
    "天齐锂业",
    "晓鸣股份",
    "长城科技",
    "东方电气"
]
candidates = [
    "龙蟠科技",
    "德方纳米",
    "欣旺达",
    "联创电子",
    "赛轮轮胎",
    "金通灵",
    "中广天择",
    "海象新材",
    "小熊电器",
    "方大集团",
    "盐津铺子",
    "涪陵榨菜",
    "苏泊尔",
    "太阳能",
    "中兵红箭",
    "长城科技",
    "惠泉啤酒",
    "天壕环境",
    "中天火箭",
    "金杯电工",
    "明阳智能"
]
await mon(holdings, candidates, arrow.now().shift(seconds=5))
print("一天监测结束")

In [2]:
say("你好")